In [1]:
import tensorflow as tf
import zipfile
tfkl = tf.keras.layers
import csv
import numpy as np
import pandas as pd
import xport
import os as os
import matplotlib.pyplot as plt
#from keras.layers import Input, Embedding, Flatten, Dense, BatchNormalization, Dropout, concatenate
#from keras.models import Model
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [7]:
# loading in USMDB data
data = []
ages = []
states = []
genders = []

with open("../data/usmdb/usmdb.csv", "r") as file:
    reader = csv.reader(file,delimiter=',')
    for row_index, row in enumerate(reader):
        if row_index == 0:
            print(row)
        if row_index >= 1:
            state, gender, year, age, rate = row
            year = int(year)
            try:
                age = int(age)
            except:
                age = -1
            if state not in states:
                states.append(state)
            state = states.index(state)
            if gender not in genders:
                genders.append(gender)
            gender = genders.index(gender)
            try:
                rate = float(rate)
            except:
                rate = -1
            if rate > 1:
                rate = 1
            if age != -1 and rate != -1 and age <= 99:
                data.append([state, gender, year, age, rate])

data = np.array(data)

['PopName', 'Sex', 'Year', 'Age', 'mx']


In [12]:
df = pd.read_csv("../data/brfss/brfss_agg.csv", index_col=0)
df['year'] = pd.to_numeric(df['year'], errors='coerce')
df['state'] = pd.to_numeric(df['state'], errors='coerce')
df['age'] = df['age'].astype('category')
df['sex'] = df['sex'].astype('category')

health_data = df.to_numpy()

print(health_data)
print(health_data.shape)

[[1 '18-24' 'female' 1993 21.70425030465645]
 [1 '18-24' 'female' 1994 23.100607781605586]
 [1 '18-24' 'female' 1995 23.313510430118065]
 ...
 [56 '80+' 'male' 2019 26.36584752016349]
 [56 '80+' 'male' 2020 26.72669220177878]
 [56 '80+' 'male' 2021 26.220133114156347]]
(37739, 5)
